In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *
import os


In [ ]:
spark = (
    SparkSession.builder
    .appName("Exercise6-Optimized")
    .master("local[*]")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
)
spark


In [ ]:
csv_files_path = "csv_files"
# Check if the folder exists
if not os.path.exists(csv_files_path):
    raise FileNotFoundError(f"Folder not found: {csv_files_path}")
else:
    print(f"Found CSV folder: {csv_files_path}")


In [ ]:
csv_files_path = "csv_files"
# Check if the folder exists
if not os.path.exists(csv_files_path):
    raise FileNotFoundError(f"Folder not found: {csv_files_path}")
else:
    print(f"Found CSV folder: {csv_files_path}")


In [ ]:
df = spark.read.csv("csv_files/Divvy_Trips_2019_Q4.csv",header=True,inferSchema=True)
print("Data loaded successfully!")

In [ ]:
df.count()

In [ ]:
df.show()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
df1 = spark.read.csv("csv_files/Divvy_Trips_2020_Q1.csv",header=True,inferSchema=True)
print("Data loaded successfully!")

In [ ]:
df.show()

In [ ]:
df1.show()

### Questions for Divvy_Trips_2019_Q4.csv file

### Q1. What are the `average` trip duration per day?

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import *

In [ ]:
def average_trip_duration_per_day(df):
    df_cleaned = df.withColumn("tripduration",regexp_replace(col("tripduration"), ",", "").cast(DoubleType()).cast(IntegerType()))

    df_with_date = df_cleaned.withColumn("date",to_date(col("start_time")))

    df_avg_per_day = df_with_date.groupBy("date").agg(avg("tripduration").alias("avg_trip_duration_raw"))
    
    df_rounded = df_avg_per_day.withColumn("avg_trip_duration",round(col("avg_trip_duration_raw"), 2)).drop("avg_trip_duration_raw")

    return df_rounded


In [ ]:
result = average_trip_duration_per_day(df)
result.show()

In [ ]:
result.write.mode("overwrite").option("header", "True").csv("reports/average_trips_per_day")

### Q2. How many trips were taken per day?

In [ ]:
df.show()

In [ ]:
def total_trips_per_day(df):
    df_with_date = df.withColumn("date", to_date(col("start_time")))

    df_total_per_day = df_with_date.groupBy("date") \
                        .agg(count("trip_id").alias("total_trips_per_day")) \
                        .orderBy("date")
    df_total_per_day.show()
    return df_total_per_day


In [ ]:
result = total_trips_per_day(df)
result.show()

In [ ]:
result.write.mode("overwrite").option("header", "True").csv("reports/total_trips_per_day")

### Q3. What was the most popular starting trip station for each month?

In [ ]:
df.show()

In [ ]:
def most_popular_station_each_month(df):
    df_month = df.withColumn("month", month(col("start_time")))
    df_count = df_month.groupBy("month", "from_station_name").agg(count("*").alias("trip_count"))
    window = Window.partitionBy("month").orderBy(col("trip_count").desc())
    df_ranked = df_count.withColumn("rank", row_number().over(window))
    df_most_popular = df_ranked.filter(col("rank") == 1).orderBy("month")
    df_most_popular.show()
    return df_most_popular

In [ ]:
result = most_popular_station_each_month(df)
result.show()

In [ ]:
df.show()

### Q4. What were the top 3 trip stations each day for the last two weeks?

Here is the logic you should use in PySpark:

Step-by-step approach

1. Clean tripduration (if required)

2. Extract date

3. Filter only last 14 days

4. Group by date + start_station

5. rank them

6. take top 3 per day

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window

def top_3_trip_count(df):
    df_cleaned = df.withColumn("tripduration",regexp_replace(col("tripduration"), ",", "").cast(DoubleType()).cast("int"))
    df_with_date = df_cleaned.withColumn("date", to_date(col("start_time")))
    max_date = df_with_date.agg(F.max("date")).first()[0]
    last_14 = df_with_date.filter(col("date") > F.date_sub(F.lit(max_date), 14))
    daily_counts = last_14.groupBy("date", "from_station_name").agg(F.count("*").alias("trip_count"))
    window_func = Window.partitionBy("date").orderBy(col("trip_count").desc())
    top3 = daily_counts.select('*', rank().over(window_func).alias('rank')).filter(col('rank') <= 3)
    return top3


In [ ]:
result_top3 = top_3_trip_count(df)
result_top3.show()

In [ ]:
df.printSchema()

In [33]:
df = df.withColumn("birth_date",F.to_date(F.concat(F.lit("01-01-"), F.col("birthyear")), "dd-MM-yyyy"))
df = df.withColumn("age",floor(datediff(current_date(), col("birth_date")) / 365.25))
df = df.withColumn("tripduration", regexp_replace(col("tripduration"), ",", "").cast("double").cast("int"))
df = df.withColumn("date", to_date(col("start_time")))

### Q5. Do `Male's` or `Female's` take longer trips on average?

In [38]:
df_gender_avg = df.filter(col("gender").isNotNull()) \
    .groupBy("gender") \
    .agg(round(avg("tripduration"), 2).alias("Average Trip Duration")) \
    .orderBy("Average Trip Duration", ascending=False)

df_gender_avg.show()

+------+---------------------+
|gender|Average Trip Duration|
+------+---------------------+
|Female|              1101.95|
|  Male|               855.31|
+------+---------------------+



### Q6. What is the top 10 ages of those that take the longest trips, and shortest?

In [ ]:
def top_10_age_long_short(df):
    df = df.withColumn("birth_date",F.to_date(F.concat(F.lit("01-01-"), F.col("birthyear")), "dd-MM-yyyy"))
    df = df.withColumn("age",floor(datediff(current_date(), col("birth_date")) / 365.25))
    df = df.withColumn("tripduration", regexp_replace(col("tripduration"), ",", "").cast("double").cast("int"))
    df = df.withColumn("date", to_date(col("start_time")))
    top10_longest = df.filter(F.col('age').isNotNull()) \
                .orderBy(F.col("tripduration").desc()) \
                .select('age', 'tripduration').limit(10)
    top10_shortest = df.filter(F.col('age').isNotNull()) \
                .orderBy(F.col("tripduration").asc()) \
                .select('age', 'tripduration').limit(10)
    return top10_longest, top10_shortest

In [ ]:
result1, result2 = top_10_age_long_short(df)
result1.show()
result2.show()